# Background inspection

Helps with creating background videos and comparing them between caiman and fiji

In [ ]:
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm
import napari


import caiman as cm


from fcutils.file_io.io import load_yaml, save_yaml
from fcutils.plotting.utils import clean_axes, save_figure, add_colorbar_to_img
from fcutils.plotting.colors import * 
from fcutils.plotting.plot_elements import plot_shaded_withline
from fcutils.file_io.utils import check_create_folder, get_file_name

from movie_visualizer import compare_videos
from utils import print_cnmfe_components, plot_components_over_image, load_fit_cnmfe
from utils import start_server, load_params,  log_cnmfe_components, load_mmap_video_caiman

c, dview, n_processes = start_server()

## Load stuff

In [ ]:
fld = 'D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p2\\19JUN26'
metadata = load_yaml(os.path.join(fld, "01_PARAMS", "analysis_metadata.yml"))


# Load model
cnm, model_filepath = load_fit_cnmfe(fld, n_processes, dview, raw=False, curated=True)

# Get video paths
vids_paths = load_yaml(os.path.join(fld, metadata['outputfld'], "video_paths.yml"))

# SELECT WHICH VIDEO TO USE
USE_VIDEO = 'transf_pw_mc'
video = vids_paths[USE_VIDEO]
print(f"Using video {USE_VIDEO} saved at {video}")


## Get background in caiman

In [ ]:
# Get video data
Yr, dims, T, images = load_mmap_video_caiman(video)

# Get video background
cnm.estimates.f = None
background = cnm.estimates.compute_background(Yr)
background = background.T.reshape((T, )+dims, order="F")

# Save as video
cnmf_background_file = os.path.join(fld, get_file_name(video)+"_cnmf_background.tif")
movie = cm.movie(background)
movie.save(cnmf_background_file)

## Get background in fiji

Steps to create background video in fiji
- save mmap to tif
- use rolling ball filter to create a background subtracted video
- subtract the background subtracted video from the video to get the background video
- save and load back in python


In [ ]:
# Save as tif
savename = os.path.join(fld, get_file_name(video)+".tif")

if not os.path.isfile(savename):
    movie = cm.movie(images)
    movie.save(savename)

print(f"Video saved as tif at: {savename}")

# now do stuff in fiji

In [ ]:
# Load back the tif after having created the background in fiji
fiji_background_bg_subtracted = cm.load(savename.replace(".tif", "background_subtracted.tif"))

# Now create the background in fiji
fiji_background = images - fiji_background_bg_subtracted

# save fiji file
fiji_background_file = savename.replace(".tif", "fiji_background.tif")
movie = cm.movie(fiji_background)
movie.save(fiji_background_file)


## Inspect both background side by side

In [ ]:
f, axarr = plt.subplots(ncols=2, figsize=(12, 8))
f.suptitle("CNFM fitted on NORMALIZED")

axarr[0].imshow(np.mean(background, axis=0), cmap="gray")
axarr[1].imshow(np.mean(fiji_background, axis=0), cmap="gray")

axarr[0].axis("off")
axarr[0].set(title="Caiman background")

axarr[1].axis("off")
axarr[1].set(title="Fiji background")

f.tight_layout()
save_figure(f, os.path.join(fld, "update", "normalized_video_background"))

In [ ]:
# Inspects videos side by side in napri

compare_videos(cnmf=cnmf_background_file, fiji=fiji_background_file, notebook=True, contrast_limits=None)

## Update video paths

In [ ]:
# Update video paths yaml file
vids_paths['transf_pw_mc_fiji_bg'] = fiji_background_file
vids_paths['transf_pw_mc_cnmf_bg'] = cnmf_background_file
save_yaml(os.path.join(fld, metadata['outputfld'], "video_paths.yml"), vids_paths)